**Testing Notebook**

*This notebook have testing purposes ONLY!*

Create the Spark object that shall be used for the tests.

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

Defines the function that converts a matrix from a list of rows format into a list of coordinates (using Tuples) and the element value itself.

In [2]:
def matCoordNotation (M):
    i, j = (len(M), len(M[0]))
    
    N = []
    for y in range(i):
        for x in range(j):
            N.append([(y,x), M[y][x]])
    
    return N

Defines the function that converts a matrix from a list of lists of coordinates (using Tuples) and the element value itself into a list of rows.

In [3]:
from operator import itemgetter

def matListNotation (M):
    maxValues = max(M,key=itemgetter(0))
    
    i = maxValues[0][0] + 1
    j = maxValues[0][1] + 1
    
    N = []
    for k in range(i):
        Z = [0] * j
        N.append(Z)
        
    for x in range(i*j):
        lin = M[x][0][0]
        col = M[x][0][1]
        N[lin][col] = M[x][1]
        
    return N

X and Y matrices in a typical list of rows format.

In [4]:
from random import randrange

def random_ints(num, lower=0, upper=9):
    ints = []
    for i in range(num):
        ints.append(randrange(lower,upper+1))
    return ints

def randomList(Lines, Columns):
    M = []
    for i in range(Lines):
            M.append(random_ints(Columns))
            
    return M

In [5]:
def rectifyReduce (reduce_output):
    M = []
    M.extend(reduce_output)
    rectified = []
    for i in range(int(len(M) / 2)):
        rectified.append([M[i], M[i+1]])
        del M[:1]
    
    return rectified

In [6]:
X = randomList(50,200)
Y = randomList(200,50)

X and Y matrices converted into coordinates notation

In [7]:
X_coord = matCoordNotation(X)
Y_coord = matCoordNotation(Y)

Creation of the RDDs for both matrices.

In [16]:
rdd_x = sc.parallelize(X_coord, 1)
rdd_y = sc.parallelize(Y_coord, 1)

1. Applying **.cartesian()** function to add, side by side, all elements of rdd_y to rdd_x.

2. Applying **.filter()** function to select only the correct elements to be used in the multiplication processes.

3. Applying **.map()** function to effectively multiply each couple of filtered elements

4. Applying **.reduceByKey()** function to sum all terms that have the same key, which is the coordinates of the matrix in this particular case.

5. Applying **.sortByKey()** function to put the elements of the matrix in an ascending order.

6. Applying **.collect()** to store the final matrix in a variable and converting it into a matrix that uses a typical list notation.

Obs: **.sortByKey()** and **.collect()** should NOT be used in the final code, because it's very expensive to be processed.

In [17]:
%%time
MAT = (
    rdd_x
        .cartesian(rdd_y)
        .filter(lambda x: x[0][0][1] == x[1][0][0])
        .map(lambda x: [(x[0][0][0], x[1][0][1]), x[0][1] * x[1][1]])
        .reduceByKey(lambda accum, n: accum + n)
        #.sortByKey()
        .collect()
)
        
MAT = matListNotation(MAT)
#print(MAT)

CPU times: user 9 ms, sys: 2.52 ms, total: 11.5 ms
Wall time: 14.3 s


In [18]:
%%time
MAT = (
    rdd_x
        .cartesian(rdd_y)
        .filter(lambda x: x[0][0][1] == x[1][0][0])
        .map(lambda x: [(x[0][0][0], x[1][0][1]), x[0][1] * x[1][1]])
        .reduceByKey(lambda accum, n: accum + n)
        .reduce(lambda x, y: x+y)
)

#print(matListNotation(rectifyReduce(MAT)))

CPU times: user 5.37 ms, sys: 4.81 ms, total: 10.2 ms
Wall time: 14.5 s


Creating a new simple matrix, converting it into a coordinates notation and using it to create a RDD object.

In [51]:
A = randomList(2000, 2000)
A_coord = matCoordNotation(A)
rdd_a = sc.parallelize(A_coord, 32)

Applying **.map()** to obtain the **Transpose** of the original matrix

In [50]:
%%time
B_T = (
    rdd_a
        .map(lambda x: [(x[0][1], x[0][0]), x[1]])
        .reduce(lambda x, y: x+y)
)

#print(matListNotation(rectifyReduce(B_T)))

CPU times: user 1.69 s, sys: 368 ms, total: 2.06 s
Wall time: 48min 1s
